In [ ]:
'''
VMP 2022-02-24: used in final report
'''

In [ ]:
# overall path to the project
path = "path/to/base"

In [ ]:
# check RAM
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 54.8 gigabytes of available RAM



In [ ]:
# basic setup
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
!pip install python-louvain

In [ ]:
from community import community_louvain
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd 
import pickle

In [ ]:
# setup variables
filenames = ["preprints_2016-07_2021-07.csv", "preprints_2015-04_2020-04.csv"] # actually June 21' and March 20' in terms of including. 
outnames = ["june21", "march20"] 
resolutions = [0.0001, 0.00025, 0.0005, 0.00075, 0.001, 0.0025, 0.005, 0.0075, 0.01, 0.025, 0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
inpath = f"{path}/DATA/collaboration/network/collaboration/"
outpath = f"{path}/DATA/collaboration/network/community/"

In [ ]:
# create network (GCC)
def create_network(inpath, filename):
  df_tmp = pd.read_csv(f"{inpath}{filename}")
  df_tmp = df_tmp[["AuthorId_x", "AuthorId_y"]].drop_duplicates() # unweighted
  G = nx.from_pandas_edgelist(df_tmp, "AuthorId_x", "AuthorId_y")
  Gcc = sorted(nx.connected_components(G), key=len, reverse=True)
  G0 = G.subgraph(Gcc[0])
  return G0

In [ ]:
# best partition
def create_partition(G, outpath, filename, resolution = 1): 
  # compute it 
  partition = community_louvain.best_partition(G, resolution = resolution)

  # write it 
  with open(f"{outpath}{filename}_res{resolution}.pickle", 'wb') as handle: 
    pickle.dump(partition, handle) 

In [ ]:
for filename, outname in zip(filenames, outnames): 
    GCC = create_network(inpath, filename)
    for resolution in resolutions: 
      create_partition(GCC, outpath, outname, resolution)